# Tutorial: Discrete Dynamic Programming

## Markov Chains

A worker’s employment dynamics obey the stochastic matrix

$$P = \begin{bmatrix}
1-\alpha & \alpha \\
\beta & 1-\beta
\end{bmatrix}$$



$$P = \begin{bmatrix}
1-\alpha & ... \\
\beta & ...
\end{bmatrix}$$


with $\alpha\in(0,1)$ and $\beta\in (0,1)$. First line corresponds to employment, second line to unemployment.

__Which is the stationary equilibrium? (choose any value for $\alpha$ and $\beta$)__

In [1]:
α = 0.2
β = 0.3
P = [1-α α; β 1-β]

2×2 Matrix{Float64}:
 0.8  0.2
 0.3  0.7

In [4]:
[1,1]'

1×2 adjoint(::Vector{Int64}) with eltype Int64:
 1  1

In [6]:
# we want μ = μ*P
# we want μ = P'*μ
# we want (P'-I)*μ = 0
# can we μ=(P'-I)\0
using LinearAlgebra: I
M = (P'-I)
Q = cat(
    M,
    [1,1]'
    ; dims=1
)
Q \ [0,0,1.0]

2-element Vector{Float64}:
 0.6
 0.3999999999999999

__In the long run, what will the the fraction $p$ of time spent unemployed? (Denote by $X_m$ the fraction of dates were one is unemployed)__

__Illustrate this convergence by generating a simulated series of length 10000 starting at $X_0=1$. Plot $X_m-p$ against $m$. (Take $\alpha=\beta=0.1$).__

## Basic Asset Pricing model

A financial asset yields dividend $(x_t)$, which follows an AR1. It is evaluated using the stochastic discount factor: $\rho_{0,t} = \beta^t \exp(y_t)$  where $\beta<1$ and $y_t$ is an $AR1$.
The price of the asset is given by $p_0 = \sum_{t\geq 0} \rho_{0,t} U(x_t)$ where $U(u)=\exp(u)^{0.5}/{0.5}$.
Our goal is to find the pricing function $p(x,y)$, which yields the price of the asset in any state.

__Write down the recursive equation which must be satisfied by $p$.__


$$p_t = U(x_t) + \beta E_t \left[ \frac{e^{y_{t+1}}}{e^{y_t}} p_{t+1} \right]$$

__Compute the ergodic distribution of $x$ and $y$.__

In [1]:
# we do it for three states
M = [ 0.9  0.05 0.05 ;
      0.05 0.9  0.05 ;
      0.05 0.05 0.9 ] 

3×3 Matrix{Float64}:
 0.9   0.05  0.05
 0.05  0.9   0.05
 0.05  0.05  0.9

In [3]:
sum(M; dims=2) # all columns sum to 1

3×1 Matrix{Float64}:
 1.0
 1.0
 1.0

In [ ]:
# we want the solution μ such that
# μ M  = μ

# or 

#M' μ = μ

In [5]:
Mp = M'

3×3 adjoint(::Matrix{Float64}) with eltype Float64:
 0.9   0.05  0.05
 0.05  0.9   0.05
 0.05  0.05  0.9

In [10]:
using LinearAlgebra
P = M'- I
# μ should satisfy (M'-I)μ = 0
# we need to add the condition $|μ|=1$

3×3 Matrix{Float64}:
 -0.1    0.05   0.05
  0.05  -0.1    0.05
  0.05   0.05  -0.1

In [15]:
P[end,:] .= 1.0
P
# we should have $ P  \mu = [0, 0, 1]
v0 = [0., 0., 1.]

3-element Vector{Float64}:
 0.0
 0.0
 1.0

In [17]:
μ = P \ v0

3-element Vector{Float64}:
 0.3333333333333333
 0.3333333333333334
 0.33333333333333326

In [23]:
μ'*M - μ'

1×3 adjoint(::Vector{Float64}) with eltype Float64:
 0.0  0.0  0.0

In [24]:
sum(μ)

1.0

__Discretize processes $(x_t)$ and $(y_t)$ using 2 states each. How would you represent the unknown $p()$?__

In [27]:
M_x = [ 0.9  0.1 ; 0.1 0.9 ]
M_y = [ 0.5  0.5 ; 0.5 0.5 ]
v_x = [-0.1, 0.1]
v_y = [-0.1, 0.1]

2-element Vector{Float64}:
 -0.1
  0.1

We represent $p(x,y)$ as a 2d matrix $(p_{ij})$ where $p_{ij}=p(x_i, y_j)$

In [28]:
P_0 = rand(2,2)

2×2 Matrix{Float64}:
 0.694689  0.349115
 0.883153  0.187633

In [ ]:
# Can we list the full states: $(x_i, y_j)$

In [43]:
import Base.Iterators: product

In [46]:
# option 1
[product(v_x, v_y)...]

4-element Vector{Tuple{Float64, Float64}}:
 (-0.1, -0.1)
 (0.1, -0.1)
 (-0.1, 0.1)
 (0.1, 0.1)

In [53]:
# option 2
grid = [(a,b) for a in v_x, b in v_y]
grid

2×2 Matrix{Tuple{Float64, Float64}}:
 (-0.1, -0.1)  (-0.1, 0.1)
 (0.1, -0.1)   (0.1, 0.1)

In [55]:
kron(M_x, M_y) 

4×4 Matrix{Float64}:
 0.45  0.45  0.05  0.05
 0.45  0.45  0.05  0.05
 0.05  0.05  0.45  0.45
 0.05  0.05  0.45  0.45

In [56]:
# can you write yourself the kronecker matrix?

__Solve for $p()$ using successive approximations__

In [58]:
# to avoid using global variables
model = merge(
    (;M_x, M_y, v_x, v_y),
    (;β=0.9)
)


(M_x = [0.9 0.1; 0.1 0.9], M_y = [0.5 0.5; 0.5 0.5], v_x = [-0.1, 0.1], v_y = [-0.1, 0.1], β = 0.9)

In [34]:
U(u) = sqrt(exp(u))/0.5

U (generic function with 1 method)

In [63]:
function evaluation_step(p_0::Matrix, model)

    (;M_x, M_y, v_x, v_y, β) = model

    p_1 = p_0*0

    #iteration over the  (current) state-space
    for i=1:size(M_x, 1)
        for j=1:size(M_y, 1)

            reward = U(v_x[i])
            
            continuation = 0.0
            
            # enumerate all future states
            for k=1:size(M_x, 1)
                for l=1:size(M_y, 1)

                    # probability of ending in (k,l) from (i,j)
                    λ = M_x[i,k] * M_y[j, l]

                    continuation += λ * p_0[k,l]

                end
            end
            
            p_1[i,j] = reward + β*continuation
        end
    end
    

    return p_1::Matrix
end

evaluation_step (generic function with 1 method)

In [67]:
function evaluation_step(p_0::Matrix, model)

    (;M_x, M_y, v_x, v_y, β) = model

    p_1 = p_0*0

    #iteration over the  (current) state-space
    for i=1:size(M_x, 1), j=1:size(M_y, 1)
            
            p_1[i,j] =  U(v_x[i]) + β*sum( M_x[i,k]*M_y[j, l] *p_0[k,l] for k=1:size(M_x, 1), l=1:size(M_y, 1) )
    end
    

    return p_1::Matrix
end

evaluation_step (generic function with 1 method)

In [68]:
evaluation_step(P_0, model)

2×2 Matrix{Float64}:
 2.37338  2.37338
 2.58318  2.58318

In [69]:
function price(model; T=100)
    p_0 = rand(2,2)
    for t=1:100
        p_1 = evaluation_step(p_0, model)
        p_0 = p_1
    end
    return p_0
end

price (generic function with 1 method)

In [70]:
price(model)

2×2 Matrix{Float64}:
 19.6672  19.6672
 20.3818  20.3818

__Solve for $p()$ by solving a linear system (homework)__

## Asset replacement (from Compecon)

At the beginning of each year, a manufacturer must decide whether to continue to operate an aging physical asset or replace it with a new one.

An asset that is $a$ years old yields a profit contribution $p(a)$ up to $n$ years, at which point, the asset becomes unsafe and must be replaced by law.

The cost of a new asset is $c$. What replacement policy maximizes profits?

Calibration: profit $p(a)=50-2.5a-2.5a^2$. Maximum asset age: 5 years. Asset replacement cost: 75, annual discount factor $\delta=0.9$.

__Define kind of problem, the state space, the actions, the reward function, and the Bellman updating equation__

kind of problem:
- discrete/finite state and actions space
- infinite horizon
- discrete dynamic programming problem (d.m.d.p.)

- state-space:  asset age $a\in[0,1,2,3,4,5]$
- actions: keep/replace ($\text{replace} in false/true$ if a<5)


Bellman updating equation:

$$ V(a) ← p(a) + \delta \begin{cases} V(0) - c \   \text{if a=true} \\ V(a+1) \text{if a=false} \\ V(0)-c \text{if a=5}\end{cases}   $$

__Solve the problem using Value Function Iteration__

In [11]:
p(a) = 50-2.5*a-2.5*a^2


p (generic function with 1 method)

In [18]:
"""
value_update: computes the Bellman update
- x: policy function
- V: Value function
"""
function value_update(x::Vector{Bool}, V::Vector{Float64}; δ=0.9, c=75.0)

    V1 = V*0

    N = length(V)
    for i=1:N

        a = i-1

        if a == 5
            # forced replacement
            # a1 = 0
            reward = p(a) - c
            contv = V[1]
        else
            if x[i]
                # replace
                reward = p(a) - c
                contv = V[1]
            else
                # not replace
                reward = p(a)
                contv = V[i+1]
            end

        end

        V1[i] = reward + δ*contv
    end

    return V1
end

value_update

In [19]:
x0 = [false, false, false, false, false]
V0 = [0.0,0,0,0,0,0]

6-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [20]:
value_update(x0, V0)

6-element Vector{Float64}:
   50.0
   45.0
   35.0
   20.0
    0.0
 -100.0

In [24]:
"""
value_update: computes the Bellman update
- x: policy function
- V: Value function
"""
function bellman_update( V::Vector{Float64}; δ=0.9, c=75.0)

    V1 = V*0
    x1 = zeros(Bool, 5) # create a vector [false, false, ...]

    N = length(V)
    for i=1:N

        a = i-1

        if a == 5
            # forced replacement
            # a1 = 0
            reward = p(a) - c
            contv = V[1]
            V1[i] = reward + δ*contv

        else
            
            # replace
            V_rep = p(a) - c + δ*V[1]
            
            # not replace
            V_nrep = p(a) + δ*V[i+1]

            if V_rep>V_nrep
                V1[i] = V_rep
                x1[i] = true
            else
                V1[i] = V_nrep
                x1[i] = false
            end

            
        end

    end

    return V1, x1
end

bellman_update

In [46]:
function vfi(V0; log=true)


    x1 = zeros(Bool, 5)

    record = [(;V=V0, x=x1)]

    for t = 1:100

        V1, x1 = bellman_update(V0)
        push!(record , (;V=V1, x=x1))
        println(x1)
        V0 = V1

    end

    return V0, x1,  record

end

vfi (generic function with 1 method)

In [47]:
using SimplePlots

In [48]:
V, x, record = vfi(V0)

Bool[0, 0, 0, 0, 0]
Bool[0, 0, 0, 0, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 1, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]
Bool[0, 0, 0, 1, 1]


([229.00165560469247, 198.8907284496583, 170.99026363319058, 151.10095881624548, 131.10095881624548, 106.10095881624548], Bool[0, 0, 0, 1, 1], NamedTuple{(:V, :x), Tuple{Vector{Float64}, Vector{Bool}}}[(V = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], x = [0, 0, 0, 0, 0]), (V = [50.0, 45.0, 35.0, 20.0, 0.0, -100.0], x = [0, 0, 0, 0, 0]), (V = [90.5, 76.5, 53.0, 20.0, -30.0, -55.0], x = [0, 0, 0, 0, 1]), (V = [118.85000000000001, 92.7, 53.0, 26.450000000000003, 6.450000000000003, -18.549999999999997], x = [0, 0, 0, 1, 1]), (V = [133.43, 92.7, 66.965, 51.965, 31.965000000000003, 6.965000000000003], x = [0, 0, 1, 1, 1]), (V = [133.43, 105.2685, 81.7685, 65.087, 45.087, 20.087000000000003], x = [0, 0, 0, 1, 1]), (V = [144.74165, 118.59165, 93.57830000000001, 65.087, 45.087, 20.087000000000003], x = [0, 0, 0, 1, 1]), (V = [156.732485, 129.22047000000003, 93.57830000000001, 75.267485, 55.26748499999999, 30.267484999999994], x = [0, 0, 0, 1, 1]), (V = [166.298423, 129.22047000000003, 102.7407365, 86.05923

In [45]:
plot([e.V[1] for e in record])

       ┌──────────────────────────────────────────────────────────────┐ 
   229 │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣀⣀⠤⠤⠒⠒⠒⠒⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉│ 
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣀⠔⠋⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⣀⡠⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⡔⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⢰⠚⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⢠⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⡤⠇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⢠⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⡸⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⢸⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⢸⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

In [44]:
plot([e.x[1]*0 for e in record])

ArgumentError: ArgumentError: height has to be positive

__Solve the problem using Policy Iteration. Compare with VFI.__

### Brock-Mirman Stochastic Growth model

This is a neoclassical growth model with unpredictable shocks on productivity.

Social planner tries to solve:

$$\max E_t \left[ \sum_{n=0}^{\infty} \beta^n \log C_{t+n} \right]$$

s.t.

$$K_{t+1} = Y_t - C_t$$
$$Y_{t+1} = A_{t+1}K_{t+1}^\alpha$$

where $A_t$ is the level of productivity in period $t$. 
It can take  values $A^h=1.05$ and $A^l=0.95$. The transition between these two states are given by the matrix:
$$P = \begin{bmatrix}
0.9, 0.1\\
0.1, 0.9
\end{bmatrix}$$

__Propose a plausible calibration__

__What are the states? What are the controls? Is it possible to bound them in a natural way? Propose a discretization scheme.__

__Write down the Bellman equation__

__How do you represent a policy function? Implement a value evaluation function.__

__Solve the model using Value Function Iteration. Plot the solution.__

__Implement Policy Improvement Steps. Compare convergence Speed.__

__Bonus: Propose some ideas to improve performances.__